In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def V(m,x,x_eq=-3,omega=0.001):
    y=m/2*omega**2*(x-x_eq)**2
    return y
def Gauss(x):
    y=np.exp(-x**2)
    return y
def k(N_G,dk):
    l=np.zeros(N_G)
    for i in range(0,N_G):
        if i<=N_G/2+1:
            l[i]=i*dk
        else:
            l[i]=-(N_G-i)*dk
    return l
def Vprop(V,dt):
    l=np.exp(-1.0j*V*dt)
    return l
def Tprop(k,m,dt):
    l=np.exp(-1.0j/(2*m)*k**2*dt)
    return l
def H(m,k,V):
    l=k**2/(2*m)+V
    return l
def Norm(f,dx):
    l=np.sum(np.conjugate(f)*f)*dx
    return l

In [ ]:
N_G=1024
Grid_min=-10
Grid_max=10
dx=(Grid_max-Grid_min)/(N_G-1)
dk=2*np.pi/(N_G*dx)

n_t=1000
t_initial=0
t_final=100000
dt=(t_final-t_initial)/(n_t-1)

m=1836
n_s=2

W_12=0.0001

In [ ]:
x_grid=np.linspace(Grid_min,Grid_max,num=N_G)
k_grid=k(N_G,dk)
t_grid=np.linspace(t_initial, t_final, n_t)
T_grid=np.zeros((n_s,n_s,N_G),dtype=np.complex128)
T_grid[0,0,:]=Tprop(k(N_G,dk),m,dt)
T_grid[1,1,:]=Tprop(k(N_G,dk),m,dt)
V_grid=np.zeros((n_s,n_s,N_G),dtype=np.complex128)
V_grid[0,0,:]=Vprop(V(m,x_grid, x_eq=-3, omega=0.001),dt/2)
V_grid[1,1,:]=Vprop(V(m,x_grid, x_eq=3, omega=0.001),dt/2)
W_grid=np.zeros((n_s,n_s,N_G),dtype=np.complex128)
W_grid[0,0,:]=np.cos(W_12*dt/4)
W_grid[0,1,:]=-1.0j*np.sin(W_12*dt/4)
W_grid[1,0,:]=-1.0j*np.sin(W_12*dt/4)
W_grid[1,1,:]=np.cos(W_12*dt/4)

In [ ]:
wf=np.zeros((n_s,N_G))
wf[0,:]=Gauss(x_grid)
wf[1,:]=0. #Gauss(x_grid)
wf/=np.sqrt(Norm(wf,dx))
print(Norm(wf[0],dx),Norm(wf[1],dx))
plt.plot(wf[0,:])
plt.plot(wf[1,:])
plt.show()

In [ ]:
wf_store=np.zeros((n_s,N_G,n_t),dtype=np.complex128)
op="ij...,j...->i..."

for i, t in enumerate(t_grid):
    #print(wf.shape)
    wf=np.einsum(op,W_grid,wf)
    wf=np.einsum(op,V_grid,wf)
    wf=np.einsum(op,W_grid,wf)
    wf=np.fft.fft(wf)
    wf=np.einsum(op,T_grid,wf)
    wf=np.fft.ifft(wf)
    wf=np.einsum(op,W_grid,wf)
    wf=np.einsum(op,V_grid,wf)
    wf=np.einsum(op,W_grid,wf)
    wf_store[:,:,i]=wf
    #print(Norm(wf,dx))
plt.imshow(np.real(np.abs(wf_store[0,:,:])**2))
plt.show()
plt.imshow(np.real(np.abs(wf_store[1,:,:])**2))
plt.show()